In [ ]:
from joblib import dump

df = pd.read_csv('result_lstm.csv')
X, y, regions, loc2idx = create_sequences(df)
X_train, X_val, X_test, y_train, y_val, y_test, y_train_s, y_val_s, scaler_y, scaler_X, regions_train, regions_val, regions_test = prepare_data(X, y, regions)

X_tr_flat = X_train.reshape(len(X_train), -1)
X_te_flat = X_test.reshape(len(X_test), -1)
reg = MultiOutputRegressor(LinearRegression())
reg.fit(X_tr_flat, y_train)
y_reg = reg.predict(X_te_flat)
lstm = build_lstm(X_train.shape[1:], y_train_s.shape[1])
es = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lstm.fit(X_train, y_train_s, validation_data=(X_val, y_val_s), epochs=5, batch_size=128, callbacks=[es], verbose=1)
y_l_s = lstm.predict(X_test)
y_l = post_process(
        y_pred_s=y_l_s,
        scaler_y=scaler_y,
    )
results = {
        'LinearReg': evaluate(y_test, y_reg),
        'LSTM': evaluate(y_test, y_l)
    }
print(pd.DataFrame(results).T.sort_values('MAE'))

dump(scaler_X, 'scaler_X.joblib')  
dump(scaler_y, 'scaler_y.joblib')  

lstm.save('lstm.keras')